## EXAMEN NLP :
### Sujet : classifieur de commentaire produit.
#### Produit : Les cinq blessures qui empêchent d'être soi-même.
#### lien : https://www.amazon.fr/cinq-blessures-emp%C3%AAchent-d%C3%AAtre-soi-m%C3%AAme/dp/2266229486/ref=cm_cr_arp_d_product_top?ie=UTF8

#### Classe : Data Scientist
#### Année : 2020 - 2021

#### Présenté par :
| Prénoms       |     Nom         |   
| ------------- |: -------------: |
| Aboubacar Sidiki        |        SIDIBE        |

GROUPE GEMA / IA-SCHOOL

# Partie 1: Collection et Nettoyage des données

## 1) Collection des commentaires produits d’amazone

### Procédures de scrapping
- a) Récupérer les commentaires positifs
- b) Récupérer les commentaires négatifs
- c) Concatener les deux datasets (commentaires positifs + commentaires négatifs)

In [2]:
#!pip install unidecode

In [3]:
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd
import numpy as np
import re
from unidecode import unidecode
import pickle

In [4]:
#### en-tête html utilisée pour la coordination entre le client (navigateur) et le serveur de open food fact, simule le comportement d'un navigateur
header = {"Accept": "image/webp,*/*",
			"Accept-Encoding": "gzip, deflate, br",
			"Accept-Language": "fr,fr-FR;q=0.8,en-US;q=0.5,en;q=0.3",
			"Connection": "keep-alive",
			"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0"
		}

In [5]:
def parse_string(value):
    """
    Parse la chaine en supprimant \n \t \r
    
    :param value : la valeur de la chaine à parser
    :type value : String
    
    :return : la chaine parsée
    :rtype : String
    """
    if value is np.nan:
        return value
    else:
        chaine = value.replace('\n', ',')
        chaine = " ".join(chaine.split()).rstrip(',').strip(',').strip()
        return chaine

##### a. les commentaires positifs

In [10]:
data_comment_positif = []
for i in range(83):
    try:
        comment_url_positive = 'https://www.amazon.fr/cinq-blessures-emp%C3%AAchent-d%C3%AAtre-soi-m%C3%AAme/product-reviews/2266229486/ref=cm_cr_arp_d_viewpnt_lft?ie=UTF8&reviewerType=all_reviews&filterByStar=positive&pageNumber={}'.format(str(i))
        res = requests.get(comment_url_positive, headers = header)
        soup = BeautifulSoup(res.text, 'html.parser')
        content = soup.findAll('div', attrs={'data-hook':'review'})
        for j in range(len(content)):
            label = content[j].findAll('a', attrs={'class':'a-link-normal'})[0].text.split(',')[0]
            comment = content[j].findAll('span',attrs={"data-hook":"review-body"})[0].text.strip()
            data_comment_positif.append([comment, label])
    except:
        continue     

In [11]:
len(data_comment_positif) #### nombre de commentaires positifs obtenus

830

In [14]:
### création du dataframe contenant les commentaires positifs
df_pos = pd.DataFrame(data_comment_positif, columns=['commentaire', 'label', ])

In [15]:
### entête du dataframe
df_pos.head()

,commentaire,label
0,"Lise Bourbeau, l'auteur de ces Cinq blessures ...",4
1,Ce livre est une révélation. Je n’au Jamais pu...,5
2,livre parfait pour commencer à se connaître et...,5
3,Si effectivement vous n’avez aucune connaissan...,4
4,Alors ok il est vrai que je me suis reconnue d...,4


In [16]:
### export to csv positif comment
df_pos.to_csv('comment_positif.csv', index=False, encoding='utf-8')

##### b. les commentaires negatifs

In [18]:
data_comment_negatif = []
for i in range(28):
    try:
        comment_url_negatif = 'https://www.amazon.fr/cinq-blessures-emp%C3%AAchent-d%C3%AAtre-soi-m%C3%AAme/product-reviews/2266229486/ref=cm_cr_arp_d_viewpnt_rgt?ie=UTF8&reviewerType=all_reviews&filterByStar=critical&pageNumber={}'.format(str(i))
        res = requests.get(comment_url_negatif, headers = header)
        soup = BeautifulSoup(res.text, 'html.parser')
        content = soup.findAll('div', attrs={'data-hook':'review'})
        for j in range(len(content)):
            label = content[j].findAll('a', attrs={'class':'a-link-normal'})[0].text.split(',')[0]
            comment = content[j].findAll('span',attrs={"data-hook":"review-body"})[0].text.strip()
            data_comment_negatif.append([comment, label])
    except:
        continue   

In [19]:
len(data_comment_negatif) #### nombre de commentaires négatifs obtenus

280

In [20]:
### création du dataframe contenant les commentaires negatif
df_neg = pd.DataFrame(data_comment_negatif, columns=['commentaire', 'label'])

In [21]:
### entête du dataframe
df_neg.head()

,commentaire,label
0,"Franchement, je me suis arrêté au chapitre 1. ...",1
1,Un livre à l'américaine qui fait très secte ! ...,1
2,"Un peu déçue...Lise Bourbeau décrit les ""5 ble...",2
3,Le contenu de ce livre est complètement délira...,1
4,J'ai trouvé le thème du livre intéressant mais...,1


In [22]:
### export to csv negatif comment
df_neg.to_csv('comment_negatif.csv', index=False, encoding='utf-8')

##### c. Concatenation des deux datasets (commentaires positifs + commentaires négatifs)

In [23]:
df = pd.concat([df_pos, df_neg], ignore_index=True) #### concatenation des deux datasets

In [24]:
df.to_csv('comments.csv', index=False, encoding='utf-8') #### sauvegarde du dataset final

## 2. Analyse exploiratoire et nettoyage des données

In [80]:
### chargement des données
df = pd.read_csv('comments.csv',encoding="utf-8")

#### a. Analyse exploiratoire

In [81]:
df.shape

(1110, 2)

In [82]:
df.head()

,commentaire,label
0,"Lise Bourbeau, l'auteur de ces Cinq blessures ...",4
1,Ce livre est une révélation. Je n’au Jamais pu...,5
2,livre parfait pour commencer à se connaître et...,5
3,Si effectivement vous n’avez aucune connaissan...,4
4,Alors ok il est vrai que je me suis reconnue d...,4


In [83]:
df.describe()

,commentaire,label
count,1110,1110
unique,1058,6
top,Très bien,5
freq,5,606


In [84]:
df[790:830].head() ### entête des lignes erronées

,commentaire,label
790,un des meilleurs livres de developpement perso...,Achat vérifié
791,Great book,Achat vérifié
792,"Il m'a plu. Toutefois, heureusement qu'il y a ...",Achat vérifié
793,Super livre,Achat vérifié
794,J'ai adoré cet ouvrage. Très utile pour l'évol...,Achat vérifié


In [85]:
len(df[790:830]) ### nombre de ligne erronées

40

In [86]:
### nettoyage des lignes erronées
df = df.drop(labels=range(790, 830), axis=0)

In [87]:
df.shape ### nouvelle taille du df après nettoyage

(1070, 2)

In [88]:
### convertion du label en numérique
df['label'] = pd.to_numeric(df['label'])

In [89]:
len(df[df['label']>=4]) #### nombre de commentaires positives

790

In [90]:
len(df[df['label']<4]) #### nombre de commentaires negatifs

280

In [91]:
df.columns ### liste des colonnes dans le df

Index(['commentaire', 'label'], dtype='object')

In [92]:
df['label'].value_counts() ### nombre de commentaire par scores

5    606
4    184
1    125
3     95
2     60
Name: label, dtype: int64

#### b. Nettoyage

#### b.1) Macro nettoyage

- Enlever les duplicats
- Enlever les textes < seuil_mot
- Equilibrer le corpus (postif / négatif)

In [93]:
### on supprime les valeurs duplicates
print(df.shape)
df.drop_duplicates(subset=['commentaire'],inplace=True)
df.shape

(1070, 2)


(1023, 2)

In [94]:
### sueillage du mot
seuil_mot=15
def nb_mot(texte):
    return len(texte.split(' '))

df['Longueur']=df['commentaire'].apply(nb_mot)
df=df[df['Longueur']>seuil_mot]

In [95]:
# Binarisation 0 (1,2,3) / 1 (4,5)
df.loc[df[df['label'].astype(int)<4].index,'label']=0
df.loc[df[df['label'].astype(int)>3].index,'label']=1

In [96]:
# Egalisateur : on prend autant de texte positif que negatif (don j'identifie le label le moins present)
m=df['label'].value_counts().min()
Comments=pd.concat([df[df['label'].astype(int)==1].sample(m),df[df['label'].astype(int)==0].sample(m)])

In [97]:
Comments['label'].value_counts() ### repartitions des commentaires après égalisation

1    250
0    250
Name: label, dtype: int64

#### b.2) Nettoyage spécifique pour de l’analyse de texte

- Stop words
- carcteres speciaux (dont ponctuation)
- normalisation (accents et minuscule)

In [104]:
def nettoyage(texte):
    #charge la liste de stop words
    with open('stop-w_fr.txt') as f:
        s_w=list(set(unidecode(f.read()).split('\n')))
    s_w.append("possible")
    s_w.append("être")
    s_w.append("faire")
    s_w.append("ca")
    
    # Chargement du lem
    lem=pickle.load(open('lemmatize_fr.pickle','rb'))
    lem['abonnement']="abonné"
    lem['abonner']="abonné"
    lem['abimes']='abime'
    
    tmp=[]
    # Normalisation : accents et minuscule
    texte=unidecode(texte.lower())
    p='[a-z]{2,}'
    final=''
    for mot in re.findall(p,texte):
        if mot not in s_w :
            try:
                if lem[mot.lower()] not in s_w:
                    final=final.strip()+' '+lem[mot.lower()]
            except:
                final=final.strip()+' '+mot.lower()
    return final

In [102]:
#### test1 du nettoyeur
nettoyage("élément à modifier chez Mr Bean")

'element modifier mr bean'

In [105]:
#### test2 du nettoyeur
nettoyage("Salut-à'tous les terriens des grandes visons simplifiées , ça va là et Coùcoù!")

'salut terrien grand vison simplifiees coucou'

In [119]:
### nettoyage des commentaires dans le df
Comments['commentaire_nettoyer']=[nettoyage(elem) for elem in Comments['commentaire']]

In [111]:
Comments.columns

Index(['commentaire', 'label', 'Longueur', 'commentaire_nettoyer'], dtype='object')

In [122]:
Comments[['commentaire', 'commentaire_nettoyer','label']] ### df avec les commentaires nettoyer

,commentaire,commentaire_nettoyer,label
300,Utilisation pour développement personnel. Exce...,utilisation developpement personnel excellent ...,1
677,livre pour tous et toutes a lire et relire pou...,livrer lire relire mieux decouvrir auteur form...,1
400,on apprend beaucoup sur soi et on peut changer...,apprendre changer choses comprendre tolerance ...,1
420,"C'est exactement ce que je recherchai, ce livr...",rechercher livrer interessant recommander comp...,1
541,Nous en avons tous ! Personne n'est parfait ! ...,parfait adorer recommander remettre question r...,1
...,...,...,...
949,J ai acheté ce livre par curiosité mais je ne ...,achete livrer curiosite retrouver adhere,0
1089,J'espérais avoir une bonne surprise avec ce li...,esperais bonne surprise livrer confirmer exist...,0
1082,Ce livre est un ramassis de conneries .. étant...,livrer ramassis connerie master psychologie ho...,0
837,Je suis déçu qu'Amazon ait mis ce livre comme ...,decu amazon mis livrer best seller lit controv...,0


In [124]:
df_final = Comments[['commentaire_nettoyer','label']] ### on ne garde que les deux colonnes

In [125]:
df_final = df_final.rename(columns={'commentaire_nettoyer': 'commentaire'}) ### on renomme la colonne 'commentaire_nettoyer' en 'commentaire' dans le df_final

In [126]:
df_final.head() ### df final

,commentaire,label
300,utilisation developpement personnel excellent ...,1
677,livrer lire relire mieux decouvrir auteur form...,1
400,apprendre changer choses comprendre tolerance ...,1
420,rechercher livrer interessant recommander comp...,1
541,parfait adorer recommander remettre question r...,1


In [127]:
### sauvegarde du dataframe final avec les données nettoyées et formatées.
df_final.to_csv('comment_clean.csv', index=False, encoding='utf-8')